In [ ]:
import numpy as np

# Part 1: Natural Invariant Parameters

In [ ]:
def rotation_matrix_to_natural_parameters(R):

    epsilon = 1e-12  # Small value to avoid division by zero

    trace_R = np.trace(R)

    phi = np.arccos((trace_R - 1) / 2.0)

    if np.abs(phi) < epsilon:  # Rotation angle close to zero
        e = np.zeros(3)  # No rotation, so axis is undefined
    else:
        e = 1 / (2 * np.sin(phi)) * np.array([
            R[2, 1] - R[1, 2],
            R[0, 2] - R[2, 0],
            R[1, 0] - R[0, 1]
        ])

    return e, phi

In [ ]:
rotation_matrix = np.array([
    [1,   0,  0],
    [0,   np.cos(np.pi/2), -np.sin(np.pi/2)],
    [0,   np.sin(np.pi/2), np.cos(np.pi/2)]
])

e, phi = rotation_matrix_to_natural_parameters(rotation_matrix)
phi_d = phi/np.pi*180
print("e=", e)
print("phi=", phi_d)

e= [1. 0. 0.]
phi= 90.0


# Part 2: Linear Parameters

In [ ]:
def rotation_matrix_to_linear_parameters(R):
    e, phi = rotation_matrix_to_natural_parameters(R)
    q0 = np.cos(phi)
    q = 0.5*np.sin(phi)*e
    return q0, q

In [ ]:
q0, q = rotation_matrix_to_linear_parameters(rotation_matrix)
print("q0=", q0)
print("q=", q)

q0= 6.123233995736766e-17
q= [0.5 0.  0. ]


# Part 3: Euler-Rodrigues Parameters

In [ ]:
def rotation_matrix_to_euler_rodrigues_parameters(R):
    e, phi = rotation_matrix_to_natural_parameters(R)
    r0 = np.cos(phi/2)
    r = np.sin(phi/2)*e
    return r0, r

In [ ]:
r0, r = rotation_matrix_to_euler_rodrigues_parameters(rotation_matrix)
print("r0=", r0)
print("r=", r)

r0= 0.7071067811865476
r= [0.70710678 0.         0.        ]


# Part 4: Handling Singularity

In the context of robotics, the Euler-Rodrigues representation (also known as the axis-angle representation) characterizes rotations using a rotation axis and an angle of rotation. Singularities within this representation manifest when the rotation angle converges toward zero or π, resulting in numerical instability or undefined behavior. This occurs because, at these singularities, the rotation axis becomes indeterminate, leading to numerical inaccuracies or computational errors.

To address singularities in the Euler-Rodrigues representation, it's crucial to detect instances where the rotation angle approaches zero or π. Handling these cases separately is necessary to maintain numerical stability. One common strategy involves employing either a Taylor expansion or a specialized formula when the rotation angle nears zero or π. This approach helps mitigate numerical errors and ensures robustness in computations.

In [3]:
def euler_rodrigues_to_rotation_matrix(params):

    r1, r2, r3, theta = params
    singularity_threshold = 1e-10
    pi_threshold = np.pi - singularity_threshold
    if np.abs(theta) < singularity_threshold:
        R = np.eye(3) + np.cross(np.array([[0, -r3, r2], [r3, 0, -r1], [-r2, r1, 0]]), np.eye(3)) + \
            np.outer(np.array([r1, r2, r3]), np.array([r1, r2, r3])) * ((1 - np.cos(theta)) / (theta ** 2))
    elif np.abs(np.pi - np.abs(theta)) < singularity_threshold:
        R = np.array([[r1*r1 + (1-r1*r1)*np.cos(theta), r1*r2*(1-np.cos(theta)) - r3*np.sin(theta), r1*r3*(1-np.cos(theta)) + r2*np.sin(theta)],
                      [r1*r2*(1-np.cos(theta)) + r3*np.sin(theta), r2*r2 + (1-r2*r2)*np.cos(theta), r2*r3*(1-np.cos(theta)) - r1*np.sin(theta)],
                      [r1*r3*(1-np.cos(theta)) - r2*np.sin(theta), r2*r3*(1-np.cos(theta)) + r1*np.sin(theta), r3*r3 + (1-r3*r3)*np.cos(theta)]])
    else:
        c = np.cos(theta)
        s = np.sin(theta)
        t = 1 - c
        R = np.array([[r1*r1*t + c, r1*r2*t - r3*s, r1*r3*t + r2*s],
                      [r1*r2*t + r3*s, r2*r2*t + c, r2*r3*t - r1*s],
                      [r1*r3*t - r2*s, r2*r3*t + r1*s, r3*r3*t + c]])
    
    return R
params_singular = np.array([0.1, 0.2, 0.3, np.pi - 1e-8])
R_singular = euler_rodrigues_to_rotation_matrix(params_singular)
print("Rotation matrix (singular case):")
print(R_singular)
params_non_singular = np.array([0.5, -0.3, 0.2, np.pi/3])
R_non_singular = euler_rodrigues_to_rotation_matrix(params_non_singular)
print("\nRotation matrix (non-singular case):")
print(R_non_singular)


Rotation matrix (singular case):
[[-0.98  0.04  0.06]
 [ 0.04 -0.92  0.12]
 [ 0.06  0.12 -0.82]]

Rotation matrix (non-singular case):
[[ 0.625      -0.24820508 -0.20980762]
 [ 0.09820508  0.545      -0.4630127 ]
 [ 0.30980762  0.4030127   0.52      ]]
